In [4]:
using CSV, LinearAlgebra, Statistics, DataFrames, Optim, ForwardDiff, Distributed, FixedEffectModels

In [5]:
versioninfo()

Julia Version 1.9.1
Commit 147bdf428c (2023-06-07 08:27 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 8 × Intel(R) Core(TM) i7-4770 CPU @ 3.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, haswell)
  Threads: 5 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 4


In [6]:
cd("C:\\Users\\Veli\\Downloads\\Code\\data\\clean")

In [7]:
data = CSV.read("NH_MA_data_2014.csv", DataFrame);

df = data[data.DEASCHEDULE .== "CII",:];

df = filter(row -> row.QTY > 0, df);

df = filter(row -> row.AGE != "90+", df);
df.AGE = string.(df.AGE)
df.AGE = parse.(Int, df.AGE);

filter!(row -> row.NH_COUNTY_CODE != 999, df)
filter!(row -> row.MEMBER_COUNTY != 999, df)
filter!(row -> row.PROPRIETARYNAME != "ENDOCET", df)
filter!(row -> row.SV_STAT == "P", df)
filter!(row -> row.AMT_BILLED != 0.0, df);

df.PRICE = df.AMT_BILLED ./ df.QTY;

In [8]:
product_ids = unique(df.PROPRIETARYNAME);

In [9]:
product_dict = Dict{String, Int64}()
for (index, value) in enumerate(product_ids)
    product_dict[value] = index
end

In [10]:
swapped_dict = Dict(value => key for (key, value) in product_dict)

Dict{Int64, String} with 40 entries:
  5  => "Hydrocodone Bitartrate and Acetaminophen"
  35 => "Hydromet"
  20 => "OXYCODONE HYDROCHLORIDE"
  16 => "Dexmethylphenidate Hydrochloride"
  12 => "Oxymorphone Hydrochloride"
  30 => "OxyContin"
  24 => "Oxycodone and Acetaminophen"
  28 => "Adderall"
  17 => "Oxymorphone hydrochloride"
  8  => "Methylphenidate Hydrochloride"
  23 => "Hydrocodone Bitartrate And Acetaminophen"
  1  => "Morphine Sulfate"
  32 => "Daytrana"
  19 => "Hydromorphone Hydrochloride"
  37 => "PERCOCET"
  22 => "Belladonna and Opium"
  11 => "Codeine sulfate"
  36 => "Opium Tincture Deodorized"
  6  => "Vyvanse"
  9  => "Dextroamphetamine saccharate, amphetamine aspartate monohydrate, dextr…
  31 => "Oxycodone Hydrochloride"
  14 => "METHADONE HYDROCHLORIDE"
  3  => "OXYCODONE AND ACETAMINOPHEN"
  39 => "Dextroamphetamine Sulfate"
  29 => "oxycodone hydrochloride"
  ⋮  => ⋮

In [11]:
df.product_id = map(x -> product_dict[x], df.PROPRIETARYNAME)
prod_id_grped = groupby(df, :product_id)
product_ids_to_drop = []
for i in 1:length(prod_id_grped)
    length = size(prod_id_grped[i],1)
    product = prod_id_grped[i].product_id[1]
    if length <= 10 
        println("Product ", swapped_dict[product], " has ", length, " claims.")
        push!(product_ids_to_drop, product)
    end
end
filter!(row -> !(row.product_id in product_ids_to_drop), df);

Product Oxymorphone Hydrochloride has 10 claims.
Product Belladonna and Opium has 3 claims.
Product Meperidine Hydrochloride has 6 claims.
Product Opium Tincture Deodorized has 10 claims.
Product Morphine sulfate has 5 claims.


In [12]:
product_ids = unique(df.PROPRIETARYNAME);

product_dict = Dict{String, Int64}()

for (index, value) in enumerate(product_ids)
    product_dict[value] = index
end

df.product_id = map(x -> product_dict[x], df.PROPRIETARYNAME)

avg_price_df = combine(groupby(df, :product_id), :PRICE => mean => :AVG_PRICE)

avg_price_dict = Dict(zip(avg_price_df.product_id, avg_price_df.AVG_PRICE))

    
# price_vector = []
# for (prod_id,price) in enumerate(avg_price_dict)
#     push!(price_vector, avg_price_dict[prod_id])
# end

other_prices_list = []
for row in eachrow(df)
    new_dict = copy(avg_price_dict)
    new_dict[row.product_id] = row.PRICE
    price_vector = []
    for (prod_id,price) in enumerate(new_dict)
        push!(price_vector, avg_price_dict[prod_id])
    end
    push!(other_prices_list, price_vector)
end

In [13]:
prices = collect(df.PRICE)

df.COINS_PERCENT = df.AMT_COINS ./ df.AMT_BILLED

coinsurance = collect(df.COINS_PERCENT)

df.FLAT_PAY = (df.AMT_COPAY + df.AMT_DEDUCT) ./ df.QTY

copay = collect(df.FLAT_PAY);

In [14]:
function choice_prob_numerator_v4(α,δ_j,price,coinsurance,copay)
    util(α, p, δ_j) = exp.(α .* p .+ δ_j)
    p = price .* coinsurance .+ copay
    utility = util(α,p,δ_j)
    return utility
end

function choice_prob_denominator_v2(α_scalar,δ_vector,price_vector,coinsurance_scalar,copay_scalar)
    util_values = choice_prob_numerator_v4.(α_scalar, δ_vector, price_vector, coinsurance_scalar, copay_scalar)
    total = sum.(util_values)
    return total
end

choice_prob_denominator_v2 (generic function with 1 method)

In [15]:
est_coefs = CSV.read("est_coefs_2014.csv", DataFrame)

Row,Adderall,CONCERTA,Codeine sulfate,DEXTROAMPHETAMINE SULFATE,Daytrana,Dexmethylphenidate Hydrochloride,Dextroamphetamine Saccharate and Amphetamine Aspartate and Dextroamphetamine Sulfate and Amphetamine Sulfate,"Dextroamphetamine Saccharate, Amphetamine Aspartate, Dextroamphetamine Sulfate and Amphetamine Sulfate",Dextroamphetamine Sulfate,"Dextroamphetamine saccharate, amphetamine aspartate monohydrate, dextroamphetamine sulfate, amphetamine sulfate",FENTANYL,FENTANYL TRANSDERMAL,Focalin,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,Hydrocodone Bitartrate And Acetaminophen,Hydrocodone Bitartrate and Acetaminophen,Hydrocodone Bitartrate and Homatropine Methylbromide,Hydrocodone Bitartrate and Ibuprofen,Hydromet,Hydromorphone Hydrochloride,METHADONE HYDROCHLORIDE,MORPHINE SULFATE,Methadone Hydrochloride,Methylphenidate Hydrochloride,Morphine Sulfate,OXYCODONE AND ACETAMINOPHEN,OXYCODONE HYDROCHLORIDE,OxyContin,Oxycodone Hydrochloride,Oxycodone and Acetaminophen,Oxycodone hydrochloride,Oxymorphone Hydrochloride,Oxymorphone hydrochloride,PERCOCET,Ritalin,Vyvanse,Year,oxycodone hydrochloride,α
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64
1,19.6051,17.2907,15.5686,16.3096,17.7373,17.5111,16.5335,21.1006,18.0837,20.0405,19.6154,16.3703,19.3495,19.5913,18.2034,19.5555,17.7584,16.8859,17.7533,19.926,19.509,18.9486,16.8643,20.8849,19.2204,20.2114,21.0806,20.4151,18.2353,19.6112,13.3901,13.842,17.8227,15.9201,17.7259,20.8541,2014,19.4409,-0.465751


In [16]:
δ_df = select(est_coefs, Not([:α,:Year]))

Row,Adderall,CONCERTA,Codeine sulfate,DEXTROAMPHETAMINE SULFATE,Daytrana,Dexmethylphenidate Hydrochloride,Dextroamphetamine Saccharate and Amphetamine Aspartate and Dextroamphetamine Sulfate and Amphetamine Sulfate,"Dextroamphetamine Saccharate, Amphetamine Aspartate, Dextroamphetamine Sulfate and Amphetamine Sulfate",Dextroamphetamine Sulfate,"Dextroamphetamine saccharate, amphetamine aspartate monohydrate, dextroamphetamine sulfate, amphetamine sulfate",FENTANYL,FENTANYL TRANSDERMAL,Focalin,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,Hydrocodone Bitartrate And Acetaminophen,Hydrocodone Bitartrate and Acetaminophen,Hydrocodone Bitartrate and Homatropine Methylbromide,Hydrocodone Bitartrate and Ibuprofen,Hydromet,Hydromorphone Hydrochloride,METHADONE HYDROCHLORIDE,MORPHINE SULFATE,Methadone Hydrochloride,Methylphenidate Hydrochloride,Morphine Sulfate,OXYCODONE AND ACETAMINOPHEN,OXYCODONE HYDROCHLORIDE,OxyContin,Oxycodone Hydrochloride,Oxycodone and Acetaminophen,Oxycodone hydrochloride,Oxymorphone Hydrochloride,Oxymorphone hydrochloride,PERCOCET,Ritalin,Vyvanse,oxycodone hydrochloride
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,19.6051,17.2907,15.5686,16.3096,17.7373,17.5111,16.5335,21.1006,18.0837,20.0405,19.6154,16.3703,19.3495,19.5913,18.2034,19.5555,17.7584,16.8859,17.7533,19.926,19.509,18.9486,16.8643,20.8849,19.2204,20.2114,21.0806,20.4151,18.2353,19.6112,13.3901,13.842,17.8227,15.9201,17.7259,20.8541,19.4409


In [17]:
setdiff(names(δ_df),unique(df.PROPRIETARYNAME))

2-element Vector{String}:
 "Oxycodone hydrochloride"
 "Oxymorphone Hydrochloride"

In [18]:
α_hat = est_coefs.α[1]

-0.46575145336857554

Need to reorder the $\delta$ vector to match indexing of the functions that will be used later.

In [19]:
δ_hat = zeros(size(δ_df,2))
for (i,j) in enumerate(product_dict)
    #println(δ_df[1,j[1]], "  ", j[2])
    δ_hat[j[2]] = δ_df[1,j[1]]
end

In [21]:
# This was required for 2014 for some reason. Must have been an error in a previous step I couldn't implement. 
#δ_hat = δ_hat[1:end-2]

35-element Vector{Float64}:
 19.220374934633316
 19.59125471971066
 20.211401960170605
 18.9486069076709
 19.555507389299628
 20.85411098676891
 19.349526359215556
 20.884871206884167
 20.040525785877147
 21.10056220092389
 15.568590817719235
 16.864310767748428
 19.508993695641568
  ⋮
 16.885939573765704
 19.605078248184306
 19.440908381738023
 20.41510092284246
 18.235269972254493
 17.73726371711646
 16.37027067296148
 17.290722176406756
 17.753309184196564
 15.920103592211023
 18.083698357376086
 16.30960982121488

In [22]:
avg_price_df = combine(groupby(df, :product_id), :PRICE => mean => :AVG_PRICE)
avg_price_dict = Dict(zip(avg_price_df.product_id, avg_price_df.AVG_PRICE))
price_vector = []
for (prod_id,price) in enumerate(avg_price_dict)
    push!(price_vector, avg_price_dict[prod_id])
end
other_prices_list = []
for row in eachrow(df)
    new_dict = copy(avg_price_dict)
    new_dict[row.product_id] = row.PRICE
    price_vector = []
    for (prod_id,price) in enumerate(new_dict)
        push!(price_vector, new_dict[prod_id])
    end
    push!(other_prices_list, price_vector)
end;

In [231]:
# δ_hat = [16.759160102440646,
#     20.108844785905873,
#     14.27283197366383,
#     20.388565499734508,
#     18.33452707834806,
#     17.64026642093988,
#     14.270707383296973,
#     14.274409774733833,
#     17.88309100731989,
#     14.274409700854104,
#     15.650850124743782,
#     16.839173501997514,
#     18.124193273511132,
#     14.273631417363939];

In [232]:
# α_hat = 0.1105473018673869

In [23]:
vector_δ_hat = fill(δ_hat,size(df,1));

In [24]:
est_numerator_list = choice_prob_numerator_v4(α_hat,δ_hat[df.product_id],prices,coinsurance,copay);

In [25]:
est_denominator_list = choice_prob_denominator_v2(α_hat,vector_δ_hat,other_prices_list,coinsurance,copay);

In [26]:
df.s_ij = est_numerator_list ./ est_denominator_list;

In [27]:
rows = length(δ_hat)
cols = length(δ_hat)
adj_matrix = zeros(Float64, rows, cols);

In [28]:
for k in 1:length(δ_hat)
    for j in 1:length(δ_hat)
            ϵ_jk = 0
            for i in 1:size(df,1)
                if df.product_id[i] == j
                    s_ik_numerator = choice_prob_numerator_v4(α_hat,δ_hat[k],other_prices_list[i][k],coinsurance[i],copay[i])
                    s_ik_denominator = choice_prob_denominator_v2(α_hat,vector_δ_hat[i],other_prices_list[i],coinsurance[i],copay[i])
                    s_ik = s_ik_numerator / sum(s_ik_denominator)
                    if j == k
                        ϵ_ijk = -α_hat * df.s_ij[i] * (1 - df.s_ij[i])    
                        ϵ_jk += ϵ_ijk
                    else
                        ϵ_ijk = -α_hat * df.s_ij[i] * s_ik    
                        ϵ_jk += ϵ_ijk
                    end
                end
            end
        #println("ϵ($j, $k): ", ϵ_jk)
        adj_matrix[j,k] = ϵ_jk
    end
end

In [29]:
adj_matrix_df = DataFrame(adj_matrix,:auto)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,4.03737,0.122878,0.227525,0.0637926,0.118422,0.418453,0.0925948,0.438338,0.187008,0.553105,0.00220053,0.00807026,0.113578,0.0185747,0.0148431,0.0202603,0.116652,0.171929,0.544812,0.00574109,0.030716,0.124257,0.019728,0.00816506,0.119867,0.105806,0.27008,0.0312758,0.018488,0.00453811,0.0117892,0.0196313,0.00300651,0.0266657,0.00458498
2,0.169714,8.00206,0.463491,0.127406,0.243064,0.809626,0.178745,0.862243,0.364414,1.12428,0.00453372,0.0168116,0.236332,0.0364236,0.028808,0.0393045,0.225397,0.355212,1.12178,0.0116459,0.0635078,0.250316,0.0410726,0.0165541,0.231742,0.218351,0.522987,0.0625314,0.0357122,0.00877479,0.022733,0.0408876,0.0058103,0.0525762,0.00927924
3,0.618579,0.936228,28.5007,0.462287,0.896682,2.93528,0.648583,3.11923,1.31893,4.1165,0.0167818,0.0629427,0.883695,0.131759,0.104349,0.142386,0.819273,1.31836,4.14995,0.0425834,0.235881,0.911627,0.153675,0.0605096,0.840396,0.809511,1.89557,0.226979,0.12955,0.0318889,0.0825267,0.153052,0.0210736,0.19023,0.0338809
4,0.0476475,0.0694537,0.128734,2.30706,0.0669539,0.239728,0.0531403,0.249456,0.106748,0.313021,0.00124375,0.00455759,0.0641471,0.01058,0.00848801,0.0115885,0.0682111,0.0971526,0.307945,0.00324984,0.0173558,0.0703986,0.0111416,0.00462225,0.0687073,0.0597935,0.154644,0.0177393,0.0106045,0.002656,0.00677315,0.0110867,0.00172383,0.0151664,0.00259611
5,0.185124,0.289261,0.517272,0.135388,8.48069,0.770976,0.168032,0.876241,0.358193,1.24843,0.00518802,0.0196819,0.276048,0.0366311,0.0278461,0.0379156,0.196003,0.408886,1.28198,0.0128701,0.0732178,0.27201,0.0480291,0.018271,0.219783,0.250745,0.500124,0.0666411,0.0337015,0.00761419,0.0212159,0.047851,0.00549847,0.053829,0.0101982
6,2.07513,3.03234,5.61327,1.57402,2.92207,91.5259,2.29797,10.8264,4.62561,13.6452,0.054305,0.19925,2.80404,0.458949,0.367517,0.501719,2.94184,4.2433,13.4446,0.141631,0.758111,3.06539,0.48706,0.201429,2.97233,2.61124,6.69253,0.771673,0.458655,0.114551,0.2928,0.484682,0.0745667,0.658458,0.113109
7,0.0979195,0.143942,0.26566,0.0740119,0.138586,0.479456,4.67565,0.505802,0.214976,0.645374,0.00257812,0.00948534,0.13345,0.0214076,0.0170325,0.023244,0.133278,0.201595,0.63818,0.00669456,0.0360239,0.144574,0.0231833,0.0095195,0.137288,0.124021,0.309581,0.0362993,0.0211652,0.00518673,0.013483,0.0230723,0.00344275,0.0307969,0.00534256
8,2.20951,3.22953,5.97789,1.67534,3.11207,11.001,2.43542,96.8258,4.91286,14.5311,0.0578362,0.212192,2.98621,0.487844,0.390063,0.532449,3.09486,4.5192,14.3189,0.150822,0.8074,3.26376,0.518699,0.214498,3.15169,2.78103,7.09944,0.821387,0.486197,0.12047,0.310175,0.516164,0.0790568,0.700278,0.120443
9,0.381251,0.553807,1.02834,0.290116,0.53416,1.93761,0.429968,2.00628,17.9378,2.50137,0.00991657,0.0362748,0.510651,0.0851564,0.0685229,0.0935679,0.556706,0.774264,2.45553,0.0259788,0.138302,0.563451,0.0886862,0.0369531,0.555513,0.476615,1.24949,0.142181,0.0857749,0.0216845,0.0548371,0.0882438,0.01394,0.121912,0.0207614


In [240]:
#CSV.write("adj_matrix_2012.csv", adj_matrix_df)

In [241]:
#adj_matrix_df = CSV.read("adj_matrix_2012.csv", DataFrame)

In [30]:
swapped_dict = Dict(value => key for (key, value) in product_dict)

Dict{Int64, String} with 35 entries:
  5  => "Hydrocodone Bitartrate and Acetaminophen"
  35 => "DEXTROAMPHETAMINE SULFATE"
  20 => "Dextroamphetamine Saccharate and Amphetamine Aspartate and Dextroamphe…
  16 => "Oxymorphone hydrochloride"
  12 => "Methadone Hydrochloride"
  30 => "FENTANYL TRANSDERMAL"
  28 => "Oxycodone Hydrochloride"
  8  => "Methylphenidate Hydrochloride"
  17 => "FENTANYL"
  24 => "Hydrocodone Bitartrate and Ibuprofen"
  1  => "Morphine Sulfate"
  19 => "OXYCODONE HYDROCHLORIDE"
  32 => "Hydromet"
  22 => "Oxycodone and Acetaminophen"
  6  => "Vyvanse"
  23 => "Hydrocodone Bitartrate and Homatropine Methylbromide"
  11 => "Codeine sulfate"
  9  => "Dextroamphetamine saccharate, amphetamine aspartate monohydrate, dextr…
  31 => "CONCERTA"
  14 => "Ritalin"
  3  => "OXYCODONE AND ACETAMINOPHEN"
  29 => "Daytrana"
  7  => "Focalin"
  25 => "Adderall"
  33 => "PERCOCET"
  ⋮  => ⋮

In [31]:
for (i, col) in enumerate(names(adj_matrix_df))
    rename!(adj_matrix_df, Symbol(col) => Symbol(swapped_dict[parse(Int, col[2:end])]))
end

In [32]:
insertcols!(adj_matrix_df, 1, :Products => names(adj_matrix_df))

Row,Products,Morphine Sulfate,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,OXYCODONE AND ACETAMINOPHEN,MORPHINE SULFATE,Hydrocodone Bitartrate and Acetaminophen,Vyvanse,Focalin,Methylphenidate Hydrochloride,"Dextroamphetamine saccharate, amphetamine aspartate monohydrate, dextroamphetamine sulfate, amphetamine sulfate","Dextroamphetamine Saccharate, Amphetamine Aspartate, Dextroamphetamine Sulfate and Amphetamine Sulfate",Codeine sulfate,Methadone Hydrochloride,METHADONE HYDROCHLORIDE,Ritalin,Dexmethylphenidate Hydrochloride,Oxymorphone hydrochloride,FENTANYL,Hydromorphone Hydrochloride,OXYCODONE HYDROCHLORIDE,Dextroamphetamine Saccharate and Amphetamine Aspartate and Dextroamphetamine Sulfate and Amphetamine Sulfate,Hydrocodone Bitartrate And Acetaminophen,Oxycodone and Acetaminophen,Hydrocodone Bitartrate and Homatropine Methylbromide,Hydrocodone Bitartrate and Ibuprofen,Adderall,oxycodone hydrochloride,OxyContin,Oxycodone Hydrochloride,Daytrana,FENTANYL TRANSDERMAL,CONCERTA,Hydromet,PERCOCET,Dextroamphetamine Sulfate,DEXTROAMPHETAMINE SULFATE
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Morphine Sulfate,4.03737,0.122878,0.227525,0.0637926,0.118422,0.418453,0.0925948,0.438338,0.187008,0.553105,0.00220053,0.00807026,0.113578,0.0185747,0.0148431,0.0202603,0.116652,0.171929,0.544812,0.00574109,0.030716,0.124257,0.019728,0.00816506,0.119867,0.105806,0.27008,0.0312758,0.018488,0.00453811,0.0117892,0.0196313,0.00300651,0.0266657,0.00458498
2,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,0.169714,8.00206,0.463491,0.127406,0.243064,0.809626,0.178745,0.862243,0.364414,1.12428,0.00453372,0.0168116,0.236332,0.0364236,0.028808,0.0393045,0.225397,0.355212,1.12178,0.0116459,0.0635078,0.250316,0.0410726,0.0165541,0.231742,0.218351,0.522987,0.0625314,0.0357122,0.00877479,0.022733,0.0408876,0.0058103,0.0525762,0.00927924
3,OXYCODONE AND ACETAMINOPHEN,0.618579,0.936228,28.5007,0.462287,0.896682,2.93528,0.648583,3.11923,1.31893,4.1165,0.0167818,0.0629427,0.883695,0.131759,0.104349,0.142386,0.819273,1.31836,4.14995,0.0425834,0.235881,0.911627,0.153675,0.0605096,0.840396,0.809511,1.89557,0.226979,0.12955,0.0318889,0.0825267,0.153052,0.0210736,0.19023,0.0338809
4,MORPHINE SULFATE,0.0476475,0.0694537,0.128734,2.30706,0.0669539,0.239728,0.0531403,0.249456,0.106748,0.313021,0.00124375,0.00455759,0.0641471,0.01058,0.00848801,0.0115885,0.0682111,0.0971526,0.307945,0.00324984,0.0173558,0.0703986,0.0111416,0.00462225,0.0687073,0.0597935,0.154644,0.0177393,0.0106045,0.002656,0.00677315,0.0110867,0.00172383,0.0151664,0.00259611
5,Hydrocodone Bitartrate and Acetaminophen,0.185124,0.289261,0.517272,0.135388,8.48069,0.770976,0.168032,0.876241,0.358193,1.24843,0.00518802,0.0196819,0.276048,0.0366311,0.0278461,0.0379156,0.196003,0.408886,1.28198,0.0128701,0.0732178,0.27201,0.0480291,0.018271,0.219783,0.250745,0.500124,0.0666411,0.0337015,0.00761419,0.0212159,0.047851,0.00549847,0.053829,0.0101982
6,Vyvanse,2.07513,3.03234,5.61327,1.57402,2.92207,91.5259,2.29797,10.8264,4.62561,13.6452,0.054305,0.19925,2.80404,0.458949,0.367517,0.501719,2.94184,4.2433,13.4446,0.141631,0.758111,3.06539,0.48706,0.201429,2.97233,2.61124,6.69253,0.771673,0.458655,0.114551,0.2928,0.484682,0.0745667,0.658458,0.113109
7,Focalin,0.0979195,0.143942,0.26566,0.0740119,0.138586,0.479456,4.67565,0.505802,0.214976,0.645374,0.00257812,0.00948534,0.13345,0.0214076,0.0170325,0.023244,0.133278,0.201595,0.63818,0.00669456,0.0360239,0.144574,0.0231833,0.0095195,0.137288,0.124021,0.309581,0.0362993,0.0211652,0.00518673,0.013483,0.0230723,0.00344275,0.0307969,0.00534256
8,Methylphenidate Hydrochloride,2.20951,3.22953,5.97789,1.67534,3.11207,11.001,2.43542,96.8258,4.91286,14.5311,0.0578362,0.212192,2.98621,0.487844,0.390063,0.532449,3.09486,4.5192,14.3189,0.150822,0.80

In [33]:
CSV.write("adj_matrix_2014.csv", adj_matrix_df)

"adj_matrix_2014.csv"